## Parte do banco de dados

In [ ]:
# Conectando ao banco de dados SQLite (criará um novo arquivo se não existir)
import sqlite3

conn = sqlite3.connect('eventos.db')
conn = sqlite3.connect('dados_eventos.db')
cursor = conn.cursor()

Consultas:

In [ ]:
# Consulta 1: Mostrar todos os eventos com suas datas, localização e tipo de evento
print("\n--- Consulta 1: Todos os eventos com detalhes ---")
cursor.execute('''
    SELECT Eventos.nome AS Evento, DadosEventos.data AS Data, DadosEventos.localizacao AS Localização, Eventos.tipo AS Tipo
    FROM Eventos
    INNER JOIN DadosEventos ON Eventos.id = DadosEventos.evento_id;
''')
resultados = cursor.fetchall()
df_resultado1 = pd.DataFrame(resultados, columns=[coluna[0] for coluna in cursor.description])
print(df_resultado1.to_markdown(index=False, numalign='left', stralign='left'))

# Consulta 2: Mostrar os dados dos 2 eventos mais próximos de iniciar
print("\n--- Consulta 2: 2 eventos mais próximos de iniciar ---")
cursor.execute('''
    SELECT Eventos.nome AS Evento, DadosEventos.data AS Data, DadosEventos.localizacao AS Localização, Eventos.tipo AS Tipo
    FROM Eventos
    INNER JOIN DadosEventos ON Eventos.id = DadosEventos.evento_id
    ORDER BY DadosEventos.data ASC
    LIMIT 2;
''')
resultados = cursor.fetchall()
df_resultado2 = pd.DataFrame(resultados, columns=[coluna[0] for coluna in cursor.description])
print(df_resultado2.to_markdown(index=False, numalign='left', stralign='left'))

# Consulta 3: Mostrar os eventos que acontecem no Rio de Janeiro
print("\n--- Consulta 3: Eventos no Rio de Janeiro ---")
cursor.execute('''
    SELECT Eventos.nome AS Evento, DadosEventos.data AS Data, DadosEventos.localizacao AS Localização, Eventos.tipo AS Tipo
    FROM Eventos
    INNER JOIN DadosEventos ON Eventos.id = DadosEventos.evento_id
    WHERE DadosEventos.localizacao LIKE '%Rio de Janeiro%';
''')
resultados = cursor.fetchall()
df_resultado3 = pd.DataFrame(resultados, columns=[coluna[0] for coluna in cursor.description])
print(df_resultado3.to_markdown(index=False, numalign='left', stralign='left'))

In [ ]:
# Fechando a conexão com o banco de dados
conn.close()